### 1. Import dei pachetti necessari


In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import speech_recognition as sr
from gtts import gTTS
import time
import playsound
import librosa
import PySimpleGUI as sg
import numpy as np
import os
import librosa
import sys
from keras.models import load_model
import cv2 as cv

### 2. Definizione delle funzioni 
- *crop*: per portrare il segnale nella dimesnione standard
- *wav2mfcc*: per estrarre dal segnale il Mel-frequency cepstral coefficients
- *main*: per generare le predizioni
- *speak*: API servita da Google per generare suono partendo dal testo

In [ ]:
def crop(input, size):
    # Elimina eventuali valori oltre il numero consentito
    output = input[0:min(size, input.shape[0])]
    # Aggiungi valori nulli per raggiungere la dimensione richiesta
    output = np.concatenate((output, np.zeros(size-output.shape[0])))
    return output


def wav2mfcc(file_path, max_len=40):
  seconds=3
  wave, sr = librosa.load(file_path, mono=True, sr=44100)
  wave = crop(wave, sr*seconds)
  mfcc = librosa.feature.mfcc(wave, sr=sr, n_fft=2048, hop_length=512, n_mfcc=40)
  return mfcc


def main(input, model):
    
    dict_comm = {"okgoogle": 0, "heysiri": 1, "alexa":2}
    
    #estraggo le features
    X = wav2mfcc(input)
    X = X.reshape(-1,40,259,1)
    
    y_ana = np.argmax(model.predict(X)[0], axis=1)[0]
    y_fede = np.argmax(model.predict(X)[1], axis=1)[0]
    y_da = np.argmax(model.predict(X)[2], axis=1)[0]

    y_va = np.argmax(model.predict(X)[3], axis=1)[0]
    vocal_assistant = list(dict_comm.keys())[y_va]
    
    #subject = str()
    
    if y_ana==0 and y_fede==0 and y_da==0:
        subject = 'Sconosciuto'
    elif y_ana==1 and y_fede==0 and y_da==0:
        subject = 'Anastasia'
    elif y_ana==0 and y_fede==1 and y_da==0:
        subject = 'Federico'
    elif y_ana==0 and y_fede==0 and y_da==1:
        subject = 'Davide'
    elif y_ana==0 and y_fede==1 and y_da==1:
        subject = 'Sconosciuto'
        
    if vocal_assistant=='alexa':
        vocal_assistant='Alexa'
    elif vocal_assistant=='okgoogle':
        vocal_assistant='Google'
    else:
        vocal_assistant='Siri'

    return subject, vocal_assistant
    

def speak(text):
    tts = gTTS(text=text, lang='it')
    filename = 'reply.mp3'
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

### 3. Demo live

In [ ]:
#importo il modello da directory
%time model = load_model('model_processing1D')

Wall time: 13.4 s


In [ ]:
duration = 3
rec_rate = 44100

cap = cv.VideoCapture(0)
status=""
f="Premere 'r' per registrare o 'Esc' per uscire"

while(True):

    r, frame = cap.read()
    cv.putText(frame, f, (15, 37), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2)
    cv.imshow('Riconoscimento identita\' soggetto e comando vocale', frame)
    
    # Registra premendo il tasto R
    if cv.waitKey(20) & 0xFF == ord('r'):
        f = 'Inizio a registrare...'
        audio = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1)
        sd.wait(3)

        write('temp.wav', rate=rec_rate, data=audio)

        subject_, vocal_assistant_ = main('temp.wav', model)
        os.remove('temp.wav')

        if subject_== 'Sconosciuto':
            output_text = "Ciao sono "+ vocal_assistant_ + ", ci conosciamo?"
        else:
            output_text = 'Ciao ' + subject_ + ", sono " + vocal_assistant_
            
        try:
            f = output_text
            speak(output_text)
        except:
            pass
        
    k = cv.waitKey(10) & 0xff
    if k == 27: # press 'ESC' to quit
        break
        
cap.release()
cv.destroyAllWindows()